In [1]:
ENV["GRB_LICENSE_FILE"] = "/Users/solvibjornsson/Gurobi/gurobi.lic"

"/Users/solvibjornsson/Gurobi/gurobi.lic"

In [2]:
using XLSX
using DataFrames

# Read data from Excel file
xf = XLSX.readxlsx("facilityData.xlsx")



function read_column(sheet, firstrow, col)
    res = []
    row = firstrow
    while !ismissing(sheet[row,col])
        push!(res,sheet[row,col])
        row+=1
    end 
    return res
end


function Customer_data(xf, sheet_name)
    sheet = xf[sheet_name]

    firstrow = 2
    col = 1
    customer = read_column(sheet, firstrow, col)

    col = 2
    latitude = read_column(sheet, firstrow, col)

    col = 3
    longitude = read_column(sheet, firstrow, col)

    col = 4
    visits = read_column(sheet, firstrow, col)

    rawdata = DataFrame(customer = customer, latitude = latitude, 
            longitude = longitude, visits = visits)
    return rawdata
end

function facilities_data(xf, sheet_name)
    sheet = xf[sheet_name]

    firstrow = 2
    col = 1
    facility = read_column(sheet, firstrow, col)

    col = 2
    latitude = read_column(sheet, firstrow, col)

    col = 3
    longitude = read_column(sheet, firstrow, col)

    col = 4
    closing_cost = read_column(sheet, firstrow, col)
    
    col = 5
    capacity = read_column(sheet, firstrow, col)

    col = 6
    type = read_column(sheet, firstrow, col)

    rawdata = DataFrame(facility = facility, latitude = latitude, 
            longitude = longitude, closing_cost = closing_cost, capacity = capacity, type = type)
    return rawdata
end


customers_data = Customer_data(xf, "Customers")

facility_data = facilities_data(xf, "Fac")


Row,facility,latitude,longitude,closing_cost,capacity,type
,Any,Any,Any,Any,Any,Any
1,Zurich West,47.3837,8.5265,20000,480,Existing
2,Geneva South,46.1793,6.3196,32500,288,Existing
3,Luzern North,47.0813,8.3017,15000,270,Existing
4,Bern North,46.9612,7.4358,75000,240,Potential
5,Lausanne,46.5152,6.6522,56250,300,Potential
6,Geneva North,46.3886,6.2428,68750,260,Potential
7,Altdorf,46.879,8.6483,62500,200,Potential
8,Montreux,46.4353,6.9167,37500,140,Potential
9,Zug,47.1525,8.5108,60000,160,Potential


# Routing Technicians within Z¨urich

In [2]:
#module Routing_Tech1
using JuMP
using Gurobi
# Call the function with your file's path and sheet name
using XLSX
using DataFrames

# Function to read Excel data into a matrix, skipping the first row and column
function read_excel_to_matrix(file_path, sheet_name)
    # Load the workbook
    wb = XLSX.readxlsx(file_path)

    # Get the specified sheet
    sheet = wb[sheet_name]

    # Read the data into an array, dropping the first row and column
    data = XLSX.getdata(sheet)#, drop=1:1, droprows=1)
    data=data[2:end,2:end]
    
    
    # Convert the array to a matrix
    #matrix_data = hcat(data...)
    return data
end

distances_matrix = read_excel_to_matrix("vrpData.xlsx", "distances")
#repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
distances_matrix




31×31 Matrix{Any}:
  0         4         9.13017  10        …   2.72029   7.65506  1.23693
  4         0         8.00999   7.2111       6.40312   4.12311  2.08087
  9.13017   8.00999   0         4.11825     11.6499   10.9636   5.53624
 10         7.2111    4.11825   0           12.7201    8.63713  5.59732
  5.38516   2.23607  10.018     8.544        7.2111    2.28035  2.20907
  5         8.06226  14.0982   14.8661   …   2.52982  10.2956   1.96977
  5.09902   7.07107  14.0769   14.2127       3.60555   8.544    1.52643
  6.32456  10.198    14.4278   16.1245       3.82099  13.2136   3.30606
 12.7279   15.8114   13.4373   17.2627      12.5936   19.9249   7.45185
  4.24264   7.61577   8.93085  11.4018       4.66905  11.6362   3.245
  ⋮                                      ⋱                      ⋮
 12.0416   13        20.7162   20.1246      10.469    12.474    4.86621
 12.2703   11.6      19.6041   18.0488      11.7661    9.55196  4.90408
 18.4391   22.3607   25.4118   27.8568      15.9812  

In [6]:
#module Routing_Tech1
using JuMP
using Gurobi
# Call the function with your file's path and sheet name
using XLSX
using DataFrames



# Function to read Excel data into a matrix, skipping the first row and column
function read_excel_to_matrix(file_path, sheet_name)
    # Load the workbook
    wb = XLSX.readxlsx(file_path)

    # Get the specified sheet
    sheet = wb[sheet_name]

    # Read the data into an array
    data = XLSX.getdata(sheet)

    # Convert the array to a matrix and remove the first row and column
    # The `[2:end, 2:end]` slicing skips the first row and the first column
    matrix_data = convert(Matrix, data)[2:end, 2:end]

    return matrix_data
end

# Replace "vrpData.xlsx" with the actual path to your Excel file
distances_matrix = read_excel_to_matrix("vrpData.xlsx", "distances")
#repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")

function vrp_load_constraint()

    #------
    # DATA
    #------

    K = 10 # number vehicles
    N = 31; # number of nodes, node 1 is the depot

    C = 6; #capa for working hours excluding driving

    d = repair_times

    c = distances_matrix
    
    #------
    # MODEL
    #------

    model = Model(Gurobi.Optimizer);

    @variable(model, x[1:N,1:N,1:K] >= 0, Bin); #node to node coverd by truck
    @variable(model, y[1:N,1:K] >= 0, Bin); #
    @variable(model, z[1:N,1:K] >= 0); # load of truck k when arriving at node i

    @objective(model, Min, sum( sum( sum( c[i,j]*x[i,j,k]  for j in 1:N) for i in 1:N) for k = 1:K)); #mimize distance covered
    

    @constraint(model,[i = 2:N], sum(y[i,k] for k in 1:K) == 1); #every cust visited
    @constraint(model,[h = 1:N, k = 1:K], sum(i == h ? 0 : x[i,h,k] for i = 1:N) == y[h,k]); #inflow = outflow
    @constraint(model,[h = 1:N, k = 1:K], sum(j == h ? 0 : x[h,j,k] for j = 1:N) == y[h,k]); #inflow = outflow
    @constraint(model,[k = 1:K], sum(d[i]*y[i,k] for i = 2:N) <= C); #not exceeding work hours capa

    @constraint(model,[i = 2:N, j = 2:N, k = 1:K], z[i,k] - d[i] >= z[j,k] - (1-x[i,j,k])*sum(d));  #load constraint - subtour elimi

    #-------
    # SOLVE
    #-------
    set_optimizer_attribute(model, "TimeLimit", 30);
    optimize!(model)
    

    println();
    for k = 1:K
        has_route = false;
        for i = 1:N
            for j = 1:N
                if (value(x[i,j,k]) == 1)
                    print(i-1,"->",j-1," ")
                    has_route = true;
                end
            end
        end
        if has_route
            println();
        end
    end

    print("\ntotal cost = ");
    println(objective_value(model));

end

vrp_load_constraint();

end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-20
Set parameter TimeLimit to value 60
Set parameter TimeLimit to value 60
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.6.0 22G436)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9660 rows, 10230 columns and 46220 nonzeros
Model fingerprint: 0x7e32c2db
Variable types: 310 continuous, 9920 integer (9920 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [6e-01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 300 rows and 320 columns
Presolve time: 0.03s
Presolved: 9360 rows, 9910 columns, 45920 nonzeros
Variable types: 300 continuous, 9610 integer (9610 binary)

Root relaxation: objective 1.131945e+02, 990 iterations, 0.02 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

ErrorException: syntax: unexpected "end"

10. Location-based heuristic

In [2]:
#module Routing_Tech1
using JuMP
using Gurobi
# Call the function with your file's path and sheet name
using XLSX
using DataFrames



# Function to read Excel data into a matrix, skipping the first row and column
function read_excel_to_matrix(file_path, sheet_name)
    # Load the workbook
    wb = XLSX.readxlsx(file_path)

    # Get the specified sheet
    sheet = wb[sheet_name]

    # Read the data into an array
    data = XLSX.getdata(sheet)

    # Convert the array to a matrix and remove the first row and column
    # The `[2:end, 2:end]` slicing skips the first row and the first column
    matrix_data = convert(Matrix, data)[2:end, 2:end]

    return matrix_data
end

# Replace "vrpData.xlsx" with the actual path to your Excel file
f = read_excel_to_matrix("vrpData.xlsx", "f_")
#repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
c_ij = read_excel_to_matrix("vrpData.xlsx", "c_ij_new")
repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")

using JuMP
using Cbc

function solve_facility_location()

    #------
    # DATA
    #------

    n = 10; 
    m = 30; # number of customers

    f = read_excel_to_matrix("vrpData.xlsx", "f_")
    c = read_excel_to_matrix("vrpData.xlsx", "c_ij_new")
    d = read_excel_to_matrix("vrpData.xlsx", "repairTimes")

    C = 6;

    #------
    # MODEL
    #------

    model = Model(Cbc.Optimizer);

    @variable(model, x[1:n] >= 0, Bin);
    @variable(model, y[1:n,1:m] >= 0, Bin);

    @objective(model, Min, sum( f[j]*x[j] for j in 1:n) + sum( sum( c[i,j]*y[i,j] for j in 1:n) for i = 1:m));

    @constraint(model,[i = 1:m], sum(y[i,j] for j in 1:n) == 1);
    @constraint(model,[j = 1:n], sum( d[i]*y[i,j] for i = 1:m) <= C*x[j]);

    #-------
    # SOLVE
    #-------
    set_time_limit_sec(model,60)
    optimize!(model)

    println();
    for j = 1:n
        allocated = false;
        for i = 1:n
            if (value(y[i,j]) >= 0.99999)
                print((i+1)," ");
                allocated = true;
            end
        end
        if allocated
            println();
        end
    end

end

solve_facility_location();

end